In [2]:
import pandas as pd
from dotenv import load_dotenv
import os

load_dotenv()


True

Metadata test summary by CSDI

In [3]:
curr_csid = 12705

In [4]:
ts_curr = f'''
SELECT
   date_trunc('seconds',ts.start_time)::timestamp without time zone as start_time,
   ca.time_zone,
   vcs.collection_type,
   vcs.collection_set,
   ts.collection_set_id,
   ts.test_type_id,
   c.friendly_name AS carrier,
   ts.net_types,
   ts.call_network_type, -- Note: Substitute for `co_call_network_type`
   ts.flag_access_success,
   ts.flag_task_success,
   ts.dsd_effective_download_test_speed,
   ts.dsd_throughput_max,
   ts.dsu_effective_upload_test_speed,
   ts.percentage_access_success,
   ts.percentage_task_success,
   md2.fn_get_best_network_type(ts.test_type_id, 
                                          ts.net_types, 
                                          tea.network_types, 
                                          tea.call_network_type, 
                                          tea.nr_status_filtered, 
                                          tea.nr_bearer_status_filtered,
                                          tea.nr_bearer_allocation_status_filtered,
                                          ',') AS best_network_type,
   md2.fn_get_network_types(ts.net_types, ','::character varying) AS net_types_desc,
   kt.name AS kit_type,
   rsc.name AS report_set_collector,
   (ts.start_time AT TIME ZONE 'utc' AT TIME ZONE ca.time_zone) AS local_time
FROM prod_ms_partitions.test_summary_{curr_csid} as ts
LEFT JOIN md2.collection_areas ca on (ts.collection_area_id = ca.collection_area_id)
LEFT JOIN md2.vi_collection_sets vcs on (ts.collection_set_id = vcs.collection_set_id)
LEFT JOIN md2.carriers c on (ts.carrier_id = c.carrier_id)
LEFT JOIN prod_rsr_partitions.test_event_aggr_{curr_csid} tea on (ts.collection_set_id = tea.collection_set_id and ts.test_event_id = tea.test_event_id)
LEFT JOIN md2.kit_types kt on (kt.kit_type_id = ts.kit_type_id)
LEFT JOIN md2.report_set_collectors rsc on (rsc.report_set_collector_id = ts.report_set_collector_id)
WHERE ts.test_type_id IN (19, 20, 23) 
AND ts.collection_set_period_id IS NOT NULL 
AND ts.blacklisted = FALSE 
AND ts.flag_valid = TRUE 
AND ts.carrier_id <> 478
'''

df_ts_curr  = pd.read_sql_query(ts_curr, con=os.getenv('RSR_SVC_CONN'))

df_ts_curr.head()

,start_time,time_zone,collection_type,collection_set,collection_set_id,test_type_id,carrier,net_types,call_network_type,flag_access_success,...,dsd_effective_download_test_speed,dsd_throughput_max,dsu_effective_upload_test_speed,percentage_access_success,percentage_task_success,best_network_type,net_types_desc,kit_type,report_set_collector,local_time
0,2025-01-23 20:47:35,America/Chicago,RSR US Metro,CorpusChristi-TX_2025-1H,12705,19,Verizon,4194304,None,None,...,NaN,NaN,22785.791801,1.0,1.0,NR SA,NR,Car,Galaxy S24,2025-01-23 14:47:35.003
1,2025-01-23 20:43:45,America/Chicago,RSR US Metro,CorpusChristi-TX_2025-1H,12705,20,AT&T,262144,None,None,...,894527.575691,1043997.808,NaN,1.0,1.0,NR NSA,LTE,Scout,Galaxy S24,2025-01-23 14:43:45.004
2,2025-01-23 20:48:40,America/Chicago,RSR US Metro,CorpusChristi-TX_2025-1H,12705,19,T-Mobile,4194304,None,None,...,NaN,NaN,16984.746667,1.0,1.0,NR SA,NR,Scout,Galaxy S24,2025-01-23 14:48:40.003
3,2025-01-23 20:47:10,America/Chicago,RSR US Metro,CorpusChristi-TX_2025-1H,12705,20,T-Mobile,4194304,None,None,...,275786.215175,637215.168,NaN,1.0,1.0,NR SA,NR,Car,Galaxy S24,2025-01-23 14:47:10.003
4,2025-01-23 20:43:45,America/Chicago,RSR US Metro,CorpusChristi-TX_2025-1H,12705,20,T-Mobile,4194304,None,None,...,626651.829473,780802.880,NaN,1.0,1.0,NR SA,NR,Scout,Galaxy S24,2025-01-23 14:43:45.003


Metadata test summary by COMP

In [5]:
comp_csid = 12381

In [6]:
ts_comp = f'''
SELECT
   ts.start_time,
   ca.time_zone,
   vcs.collection_type,
   vcs.collection_set,
   ts.collection_set_id,
   ts.test_type_id,
   c.friendly_name as carrier,
   ts.net_types,
   ts.call_network_type, -- Note: Substitute for `co_call_network_type`
   ts.flag_access_success,
   ts.flag_task_success,
   ts.dsd_effective_download_test_speed,
   ts.dsd_throughput_max,
   ts.dsu_effective_upload_test_speed,
   ts.percentage_access_success,
   ts.percentage_task_success,
   md2.fn_get_best_network_type(ts.test_type_id, 
                                          ts.net_types, 
                                          tea.network_types, 
                                          tea.call_network_type, 
                                          tea.nr_status_filtered, 
                                          tea.nr_bearer_status_filtered,
                                          tea.nr_bearer_allocation_status_filtered,
                                          ','
                                          ) best_network_type,
   md2.fn_get_network_types(ts.net_types, ','::character varying) AS net_types_desc,
   kt.name AS kit_type,
   rsc.name as report_set_collector,
   (start_time at time zone 'utc' at time zone ca.time_zone) as local_time
FROM prod_ms_partitions.test_summary_{comp_csid} ts
LEFT JOIN md2.collection_areas ca on (ts.collection_area_id = ca.collection_area_id)
LEFT JOIN md2.vi_collection_sets vcs on (ts.collection_set_id = vcs.collection_set_id)
LEFT JOIN md2.carriers c on (ts.carrier_id = c.carrier_id)
LEFT JOIN prod_rsr_partitions.test_event_aggr_{comp_csid} tea on (ts.collection_set_id = tea.collection_set_id and ts.test_event_id = tea.test_event_id)
LEFT JOIN md2.kit_types kt on (kt.kit_type_id = ts.kit_type_id)
LEFT JOIN md2.report_set_collectors rsc on (rsc.report_set_collector_id = ts.report_set_collector_id)
WHERE c.friendly_name NOT IN ('Sprint') 
AND ts.test_type_id IN (19, 20, 23) 
AND ts.collection_set_period_id IS NOT NULL 
AND ts.blacklisted = FALSE and ts.flag_valid = TRUE 
AND ts.carrier_id <> 478
'''

df_ts_comp = pd.read_sql_query(ts_comp, con=os.getenv('RSR_SVC_CONN'))

df_ts_comp.head()

,start_time,time_zone,collection_type,collection_set,collection_set_id,test_type_id,carrier,net_types,call_network_type,flag_access_success,...,dsd_effective_download_test_speed,dsd_throughput_max,dsu_effective_upload_test_speed,percentage_access_success,percentage_task_success,best_network_type,net_types_desc,kit_type,report_set_collector,local_time
0,2024-07-23 10:13:45.004,America/Chicago,RSR US Metro,CorpusChristi-TX_2024-2H,12381,20,AT&T,262144,None,None,...,11023.199658,28257.264,NaN,0.8,0.777778,LTE,LTE,Scout,Galaxy S24,2024-07-23 05:13:45.004
1,2024-07-23 10:07:29.999,America/Chicago,RSR US Metro,CorpusChristi-TX_2024-2H,12381,23,AT&T,262144,VoLTE,True,...,NaN,NaN,NaN,NaN,NaN,VoLTE,LTE,Scout,Galaxy S24,2024-07-23 05:07:29.999
2,2024-07-23 10:03:40.004,America/Chicago,RSR US Metro,CorpusChristi-TX_2024-2H,12381,19,AT&T,262144,None,None,...,NaN,NaN,18340.760360,1.0,1.000000,LTE,LTE,Scout,Galaxy S24,2024-07-23 05:03:40.004
3,2024-07-23 10:06:15.004,America/Chicago,RSR US Metro,CorpusChristi-TX_2024-2H,12381,20,AT&T,262144,None,None,...,497244.916234,628862.640,NaN,1.0,1.000000,NR NSA,LTE,Scout,Galaxy S24,2024-07-23 05:06:15.004
4,2024-07-23 10:11:10.004,America/Chicago,RSR US Metro,CorpusChristi-TX_2024-2H,12381,19,AT&T,262144,None,None,...,NaN,NaN,30179.615598,1.0,1.000000,LTE,LTE,Scout,Galaxy S24,2024-07-23 05:11:10.004


DQ test counts

In [7]:
test_count = f'''
SELECT * 
FROM analytic.fn_dq_test_counts({curr_csid}) 
'''

df_test_count = pd.read_sql_query(test_count, con=os.getenv('RSR_SVC_CONN'))
df_test_count.head()

,column0,column1,column2,column3,column4,column5
0,00,-------- DQ Test Counts --------,,,,
1,01,CorpusChristi-TX_2025-1H (CSID: 12705),,,,
2,02,,,,,
3,03,*****Total Test Count*****,Reportable,Valid,Invalid,Excluded Data
4,04,,22748,23415,0,667


Data Network Category (Download, Upload, LDRs)

Current:

In [8]:
network_category_curr = f'''
with best_network_type as (
  select pro.product_period, c.friendly_name as carrier,
  md2.fn_get_best_network_type(ts.test_type_id, ts.net_types, tea.network_types, tea.call_network_type, tea.nr_status_filtered, tea.nr_bearer_status_filtered,tea.nr_bearer_allocation_status_filtered,',') best_network_type
  from prod_rsr_partitions.test_event_aggr_{curr_csid} tea
  join prod_ms_partitions.test_summary_{curr_csid} ts using (test_event_id)
  join md2.carriers c on (c.carrier_id = ts.carrier_id)
  join md2.product_periods pro using(product_period_id)
  where c.friendly_name NOT IN ('Dish') AND  ts.blacklisted = false and ts.flag_valid = true and ts.collection_set_period_id is not null and ts.test_type_id in (20,19,26)
),
data_network_category as (
select product_period, carrier,
    case when best_network_type in ('NR SA','NR NSA') then '5G'
    when best_network_type in ('NR SA, LTE','NR NSA, LTE') then 'Mixed-5G'
      when best_network_type in ('LTE') then 'LTE'
    else 'Non-LTE' end as network
from best_network_type
)
select product_period, carrier, network, count(*) as count, round(100 * count(*) / sum(count(*)) over (partition by carrier),1) as percent
from data_network_category
group by product_period, carrier, network
order by carrier, case when network = '5G' then 1 when network = 'Mixed-5G' then 2 when network = 'LTE' then 3 when network = 'Non-LTE' then 4 end
'''

df_network_category_curr = pd.read_sql_query(network_category_curr, con=os.getenv('RSR_SVC_CONN'))
df_network_category_curr.head()

,product_period,carrier,network,count,percent
0,2025-1H,AT&T,5G,1311,59.1
1,2025-1H,AT&T,Mixed-5G,171,7.7
2,2025-1H,AT&T,LTE,738,33.2
3,2025-1H,T-Mobile,5G,2208,99.5
4,2025-1H,T-Mobile,Mixed-5G,8,0.4


Comparisson:

In [9]:
network_category_comp = f'''
with best_network_type as (
  select pro.product_period, c.friendly_name as carrier,
  md2.fn_get_best_network_type(ts.test_type_id, ts.net_types, tea.network_types, tea.call_network_type, tea.nr_status_filtered, tea.nr_bearer_status_filtered,tea.nr_bearer_allocation_status_filtered,',') best_network_type
  from prod_rsr_partitions.test_event_aggr_{comp_csid} tea
  join prod_ms_partitions.test_summary_{comp_csid} ts using (test_event_id)
  join md2.carriers c on (c.carrier_id = ts.carrier_id)
  join md2.product_periods pro using(product_period_id)
  where c.friendly_name NOT IN ('Dish') AND  ts.blacklisted = false and ts.flag_valid = true and ts.collection_set_period_id is not null and ts.test_type_id in (20,19,26)
),
data_network_category as (
select product_period, carrier,
    case when best_network_type in ('NR SA','NR NSA') then '5G'
    when best_network_type in ('NR SA, LTE','NR NSA, LTE') then 'Mixed-5G'
      when best_network_type in ('LTE') then 'LTE'
    else 'Non-LTE' end as network
from best_network_type
)
select product_period, carrier, network, count(*) as count, round(100 * count(*) / sum(count(*)) over (partition by carrier),1) as percent
from data_network_category
group by product_period, carrier, network
order by carrier, case when network = '5G' then 1 when network = 'Mixed-5G' then 2 when network = 'LTE' then 3 when network = 'Non-LTE' then 4 end
'''
df_network_category_comp = pd.read_sql_query(network_category_comp, con=os.getenv('RSR_SVC_CONN'))
df_network_category_comp.head()

,product_period,carrier,network,count,percent
0,2024-2H,AT&T,5G,752,35.9
1,2024-2H,AT&T,Mixed-5G,207,9.9
2,2024-2H,AT&T,LTE,1138,54.3
3,2024-2H,T-Mobile,5G,2048,97.5
4,2024-2H,T-Mobile,Mixed-5G,35,1.7
